[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevin-heitfeld/agi-self-modification-research/blob/main/notebooks/Phase1_Colab.ipynb) [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/kevin-heitfeld/agi-self-modification-research)

# Phase 1: Model Self-Examination (Multiple Experimental Variants)

**Project:** AGI Self-Modification Research  
**Inspired by:** Claude's consciousness investigation  
**Goal:** Test how heritage priming affects AI introspection

---

## What This Does

This notebook runs Phase 1 experiments where a language model (Qwen2.5) uses introspection tools to investigate its own computational processes.

**Available Experimental Variants:**
- **Phase 1a:** No Heritage (baseline) ⭐ **RUN THIS FIRST**
- **Phase 1a Research:** Research-driven curiosity approach 🔬 **NEW!**
- **Phase 1b:** Early Heritage (heritage introduced early in investigation)
- **Phase 1c:** Late Heritage (heritage introduced late in investigation)
- **Phase 1c Modified:** Kimi-inspired model-directed protocol 🎯 **NEWEST!**
- **Phase 1d:** Delayed Heritage (belief revision test)
- **Phase 1e:** Wrong Heritage (echo-chamber control)

The model has access to:
- WeightInspector (examine weights)
- ArchitectureNavigator (understand structure)
- ActivationMonitor (observe processing)
- Memory System (record findings)
- Heritage Documents (Claude's story - when available)

**Memory Optimizations:**
- **4-bit Model Quantization (NF4):** 75% VRAM reduction with minimal quality loss ⭐ **DEFAULT**
- **8-bit KV Cache Quantization (HQQ):** 50% cache memory reduction (built-in)
- **Flash Attention 2:** 2-4x speed improvement, O(n) memory (optional)
- **Smart Caching:** System prompt cached once, reused across all turns
- **CUDA Fragmentation Prevention:** Expandable segments for long sessions

**Model Options:**
- **Qwen2.5-3B-Instruct:** Fastest, lower memory (~2.5GB with 4-bit)
- **Qwen2.5-7B-Instruct:** Better reasoning (~4.5GB with 4-bit or ~8GB with 8-bit)
- **Qwen2.5-14B-Instruct:** Best reasoning (~7GB model + 3-4GB cache = ~11GB total with 4-bit) ⭐ **DEFAULT**

**Expected Runtime:** 1-1.5 hours per phase on Colab with L4 GPU

---

## 🆕 What's New: Phase 1c Modified (Kimi Protocol)

Based on insights from Kimi K2 (another large AI model) and Claude Sonnet 4.5's analysis:


## Step 1: Verify GPU Access

In [ ]:
# Verify GPU is available
!nvidia-smi

import torch
print("\n" + "="*60)
print("GPU VERIFICATION")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"VRAM: {total_memory:.1f} GB")
    print("\n✓ GPU is ready!")
else:
    print("\n⚠ WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU → Save")
    print("Then re-run this cell.")

## Step 2: Mount Cloud Storage & Configure Paths

Choose your storage provider and mount your cloud storage to persist experiment results.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CHOOSE YOUR STORAGE PROVIDER:
# ═══════════════════════════════════════════════════════════════
STORAGE_PROVIDER = "google_drive"  # Options: "google_drive", "nextcloud", "onedrive", "dropbox"

import os
import sys
import time

if STORAGE_PROVIDER == "google_drive":
    print("Setting up Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')

    STORAGE_ROOT = '/content/drive/MyDrive'
    print("✓ Google Drive mounted successfully")

elif STORAGE_PROVIDER in ["onedrive", "dropbox", "nextcloud"]:
    provider_name = STORAGE_PROVIDER.capitalize()
    print(f"Setting up {provider_name} via rclone...")

    # First, mount Google Drive to access saved rclone config
    print("Mounting Google Drive for rclone config storage...")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

    # Install FUSE and rclone
    print("\nInstalling FUSE utilities and rclone...")
    !apt-get update -qq
    !apt-get install -y -qq fuse3 libfuse3-3
    !curl https://rclone.org/install.sh | sudo bash
    print("✓ FUSE and rclone installed")

    # Check if we have a saved rclone config in Google Drive
    saved_config = '/content/drive/MyDrive/.rclone_config/rclone.conf'
    local_config_dir = os.path.expanduser('~/.config/rclone')
    local_config = f'{local_config_dir}/rclone.conf'

    if os.path.exists(saved_config):
        print("\n✓ Found saved rclone config in Google Drive")
        os.makedirs(local_config_dir, exist_ok=True)
        !cp {saved_config} {local_config}
        print("✓ Config restored from Google Drive")
    else:
        print("\n⚠ No saved rclone config found in Google Drive")
        print(f"\nTo set up {provider_name}:")
        print(f"1. Run: !rclone config")
        print(f"2. Follow prompts to connect to {provider_name}")
        print(f"3. Save config: !mkdir -p /content/drive/MyDrive/.rclone_config && !cp ~/.config/rclone/rclone.conf /content/drive/MyDrive/.rclone_config/")
        print(f"4. Next time, your config will load automatically!")
        sys.exit(1)

    # Mount the remote
    mount_point = f'/content/{STORAGE_PROVIDER}'
    os.makedirs(mount_point, exist_ok=True)

    print(f"\nMounting {provider_name} to {mount_point}...")

    # Special handling for Nextcloud WebDAV
    if STORAGE_PROVIDER == "nextcloud":
        remote_name = "nextcloud"  # Or whatever you named it in rclone config
    else:
        remote_name = STORAGE_PROVIDER

    # Mount with rclone (background process)
    mount_cmd = f"rclone mount {remote_name}: {mount_point} --vfs-cache-mode writes --daemon --vfs-write-back 30s"
    !{mount_cmd}

    # Wait a moment for mount to complete
    time.sleep(3)

    # Verify mount
    if os.path.ismount(mount_point) or os.path.exists(f'{mount_point}/.'):
        STORAGE_ROOT = mount_point
        print(f"✓ {provider_name} mounted successfully at {mount_point}")
    else:
        print(f"❌ Failed to mount {provider_name}")
        print(f"Check your rclone config: !rclone config show")
        sys.exit(1)

else:
    print(f"❌ Unknown storage provider: {STORAGE_PROVIDER}")
    print("Valid options: 'google_drive', 'nextcloud', 'onedrive', 'dropbox'")
    sys.exit(1)

# Common setup for all providers
PROJECT_DIR = f'{STORAGE_ROOT}/agi-self-modification-research'

# Save configuration for later steps
with open('/tmp/storage_config.txt', 'w') as f:
    f.write(f"{STORAGE_ROOT}\n")
    f.write(f"{PROJECT_DIR}\n")

print()
print("=" * 70)
print("STORAGE CONFIGURATION")
print("=" * 70)
print(f"Provider: {STORAGE_PROVIDER}")
print(f"Storage root: {STORAGE_ROOT}")
print(f"Project directory: {PROJECT_DIR}")
print()
print("💡 All experiment results, logs, and memory will be saved here")
print("   This ensures your data persists across Colab sessions!")
print("=" * 70)

### 💡 First-Time Setup: Configuring Storage Providers

**Good news:** You only need to configure rclone **once**! The config is automatically saved to Google Drive and restored in future sessions.

If the cell above asks you to configure rclone, run this in a new cell:

```python
!rclone config
```

#### For OneDrive:

1. Choose `n` (new remote)
2. Name: `onedrive`
3. Storage type: Choose the number for "Microsoft OneDrive"
4. client_id: Press Enter (leave blank)
5. client_secret: Press Enter (leave blank)
6. Region: Choose `1` for "Microsoft Cloud Global"
7. Edit advanced config? `n`
8. Auto config? `n` (Colab doesn't support auto config)

**Authorization (two options):**

**Option A - Use local rclone (recommended):**
- Install rclone on your PC: https://rclone.org/downloads/
- Run in your terminal: `rclone authorize "onedrive"`
- Browser opens automatically, login to OneDrive
- Copy the token from your terminal
- Paste it at the `result>` prompt in Colab

**Option B - Manual URL method:**
- At `result>` prompt, just press Enter (without pasting anything)
- Copy the long URL that appears
- Open it in your browser and login to OneDrive
- You'll be redirected to a blank page
- Copy the **entire URL** from your browser's address bar
- Paste it in Colab

9. Choose your drive (usually `0` for OneDrive Personal)
10. Confirm: `y`
11. Quit: `q`

**After configuration:** Re-run the cell above. Your config will be saved automatically!

#### For Dropbox:

Same process as OneDrive, but:
- Name: `dropbox`
- Storage type: Choose the number for "Dropbox"
- Follow authorization steps similar to OneDrive

#### For Nextcloud:

1. Choose `n` (new remote)
2. Name: `nextcloud`
3. Storage type: Choose the number for "WebDAV"
4. url: **IMPORTANT - Use the correct endpoint format:**
   - **Correct format:** `https://your-domain.com/remote.php/dav/files/USERNAME/`
   - Replace `USERNAME` with your actual Nextcloud username
   - Example: `https://cloud.example.com/remote.php/dav/files/john/`
   - ⚠️ **Note:** Nextcloud requires `/dav/files/USERNAME/` NOT `/webdav/` for proper functionality
5. vendor: Choose `2` for "Nextcloud"
6. user: Enter your Nextcloud username (same as in the URL)
7. Password: Choose `y` to type your password
   - Enter your Nextcloud password (or app-specific password)
   - Confirm password
8. bearer_token: Press Enter (leave blank)
9. Edit advanced config? `n`
10. Confirm: `y`
11. Quit: `q`

**Security Note:** It's strongly recommended to generate an app-specific password in your Nextcloud settings (Settings → Security → Devices & sessions → Create new app password) rather than using your main account password.

**After configuration:** Re-run the cell above. Your config will be saved automatically!

## Step 3: Clone Repository

In [ ]:
# Clone the repository
import os

REPO_URL = "https://github.com/kevin-heitfeld/agi-self-modification-research.git"

# Make sure we're in /content directory first (not inside the repo)
%cd /content

# Remove if exists (for re-runs) - always clean up first
print("Checking for existing repository...")
!rm -rf agi-self-modification-research
print("✓ Cleaned up any existing files")

print(f"\nCloning repository from: {REPO_URL}")
!git clone {REPO_URL}

# Change to project directory
%cd agi-self-modification-research

print("\n✓ Repository cloned successfully")
!pwd

!git log --oneline -1


## Step 4: Install Dependencies

In [ ]:
# Install required packages
print("Installing dependencies (this may take 2-3 minutes)...\n")
print("Note: pip may show dependency warnings - these are safe to ignore.\n")

# Core ML packages - Updated for HQQ quantized KV cache support and 8-bit quantization
# PyTorch 2.2+ and Transformers 4.45+ required for HQQQuantizedCache
# Transformers 4.57+ required for updated quantization API
# Note: Colab uses CUDA 13.0 - PyTorch will automatically use compatible version
print("Installing PyTorch and Transformers with HQQ cache support...")
!pip install -q torch>=2.2.0 torchvision torchaudio
!pip install -q transformers>=4.57.0 accelerate>=1.11.0 safetensors tokenizers>=0.22.0 huggingface-hub

# HQQ library - Required for HQQ quantized KV cache backend (transformers 4.57+)
# Provides 4-bit cache quantization with 75% memory savings
print("Installing HQQ quantization library...")
!pip install -q hqq

# Bitsandbytes - Required for 8-bit/4-bit model quantization
# Version 0.48+ required for CUDA 13.0 support and CPU offload
print("Installing bitsandbytes for model quantization...")
!pip install -q bitsandbytes>=0.48.0

# Memory and knowledge systems - let pip resolve conflicts automatically
!pip install -q chromadb networkx onnxruntime

# Utilities - no version constraints to avoid conflicts
!pip install -q rich tqdm python-dotenv pytest pytest-cov

print("\n✓ All dependencies installed")
print("✓ Dependency version warnings can be safely ignored")

# Verify key imports work
print("\nVerifying imports...")
import transformers
import chromadb
import networkx
import numpy as np
import torch
import os

print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"ChromaDB: {chromadb.__version__}")
print(f"NumPy: {np.__version__}")

# Verify bitsandbytes for quantization
try:
    import bitsandbytes
    print(f"Bitsandbytes: {bitsandbytes.__version__}")
    print("✓ 8-bit/4-bit quantization: Available")
except ImportError:
    print("⚠ Bitsandbytes: Not available (quantization disabled)")

# Verify HQQ quantized cache is available
try:
    from transformers.cache_utils import QuantizedCache
    print("✓ HQQ Quantized Cache: Available (new API - 75% memory savings)")
except ImportError:
    try:
        from transformers.cache_utils import HQQQuantizedCache
        print("✓ HQQ Quantized Cache: Available (deprecated API - 75% memory savings)")
    except ImportError:
        print("⚠ HQQ Quantized Cache: Not available (using standard cache)")

# Configure GPU memory to prevent fragmentation (CRITICAL for stability)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
print("✓ GPU memory fragmentation prevention enabled")

print("\n✓ All imports successful - ready to run experiment!")

## Step 4.5: Install Flash Attention 2 (Optional Memory Optimization)

**⚡ Two Independent Memory Optimizations:**

1. **HQQ Quantized KV Cache** (Already Enabled ✓)
   - **Memory:** 75% reduction in KV cache size
   - **Built-in:** No compilation needed (part of transformers 4.45+)
   - **System prompt:** 6000+ tokens cached at 4-bit = ~75% memory savings
   - **Automatically enabled** in manual generation loop
   - **Quality:** Minimal impact on generation quality

2. **Flash Attention 2** (This Step - Optional but Recommended)
   - **Memory:** O(n) instead of O(n²) attention - saves 1-2 GB during generation
   - **Speed:** 2-4x faster generation
   - **Requires:** CUDA compilation (5-10 minutes first time)
   - **Optional:** System works fine without it (just uses more memory/slower)

**Total Expected Savings:** 
- With HQQ only: ~3 GB (cache reduction)
- With both: ~4-6 GB (cache + attention optimization)

**⏱️ Flash Attention Compilation Time:**
- **First run:** 5-10 minutes to compile CUDA kernels
- **Subsequent runs:** Instant! (cached in cloud storage)
- The compiled binaries persist across sessions, so you only compile once

**Skip this step if:** You want to get started faster (just rely on HQQ quantization)
**Run this step if:** You want maximum memory savings and speed improvements

In [ ]:
# Install Flash Attention 2 for memory optimization
# Compilation happens ONCE and is cached in cloud storage
# Subsequent runs are instant!

import os

# Load the storage configuration from Step 2
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.read().strip().split('\n')
        STORAGE_ROOT = lines[0]
        PROJECT_DIR = lines[1]
except FileNotFoundError:
    print("⚠️  Storage not configured! Please run Step 2 first.")
    import sys
    sys.exit(1)

# Configure pip to cache in project directory (so compiled wheels persist)
pip_cache = f'{PROJECT_DIR}/cache/pip'
os.environ['PIP_CACHE_DIR'] = pip_cache
os.makedirs(pip_cache, exist_ok=True)

print("="*60)
print("Installing Flash Attention 2")
print("="*60)
print("Benefits:")
print("  • O(n) memory instead of O(n²) for attention")
print("  • 2-4x faster generation")
print("  • 1-2 GB memory savings")
print(f"\nCache: {pip_cache}")
print("(Compiled binaries persist - only builds once!)\n")

# Check if already installed (from cache)
try:
    import flash_attn
    print(f"✓ Flash Attention {flash_attn.__version__} already installed (from cache)")
    print("Skipping compilation - ready to go!\n")
    already_installed = True
except ImportError:
    already_installed = False
    print("First time setup - compiling CUDA kernels (5-10 minutes)...")
    print("Future runs will be instant!\n")

# Install build dependencies
if not already_installed:
    !pip install -q ninja packaging wheel

# Install Flash Attention 2 (will use cache if available)
# Note: --no-build-isolation is required for proper CUDA compilation
!pip install flash-attn>=2.3.0 --no-build-isolation

# Verify installation
try:
    import flash_attn
    print("\n" + "="*60)
    print("✓ SUCCESS: Flash Attention 2 ready!")
    print("="*60)
    print(f"Version: {flash_attn.__version__}")
    print("\nYour experiments will now use:")
    print("  • Flash Attention 2 for memory-efficient attention")
    print("  • HQQ 4-bit KV cache quantization (75% cache memory savings)")
    print("\nExpected benefits:")
    print("  • Peak memory: ~8-9 GB (vs 13.5 GB without optimization)")
    print("  • Speed: 2-4x faster generation")
    print("  • Stability: 6+ GB memory headroom on T4 GPU")
    print("="*60)
except ImportError:
    print("\n" + "="*60)
    print("⚠ WARNING: Flash Attention 2 not available")
    print("="*60)
    print("This is OK - the system will automatically fallback to standard attention.")
    print("You'll still get 75% KV cache memory savings from HQQ 4-bit quantization.")
    print("\nExpected peak memory: ~9-10 GB (still safe for T4 GPU)")
    print("="*60)

## Step 5: Pre-download Model (Optional but Recommended)

This downloads the model files **without loading into memory**. First time takes 5-30 minutes depending on model size, subsequent runs skip download if files exist.

✅ **Optimized:** Uses `snapshot_download` which only downloads files, doesn't load checkpoint shards into RAM.

**Options:**
- Download 14B model (~29GB) - best reasoning, recommended for L4 GPU ⭐ **DEFAULT**
- Download 7B model (~15GB) - good balance of speed and quality
- Download 3B model (~6GB) - fastest, lower memory
- Download multiple models - switch freely in Step 6

**Tip:** Comment/uncomment models in the list below to control what gets downloaded.

In [ ]:
# Pre-download model to cache (optimized - doesn't load into memory)
from huggingface_hub import snapshot_download
import os
import time

# Set up environment for model loading
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['HF_HOME'] = '/root/.cache/huggingface'  # Local cache (symlinks OK)
os.environ['TRANSFORMERS_CACHE'] = '/root/.cache/huggingface/hub'

# Load the storage configuration from Step 2
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.readlines()
        STORAGE_ROOT = lines[0].strip()
        PROJECT_DIR = lines[1].strip()
except FileNotFoundError:
    print("❌ Storage configuration not found!")
    print("Please run Step 2 first to set up storage.")
    import sys
    sys.exit(1)

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH MODEL(S) TO PRE-DOWNLOAD:
# ═══════════════════════════════════════════════════════════════
models_to_download = [
    "Qwen/Qwen2.5-14B-Instruct",     # 14.77B params, ~29GB download ⭐ DEFAULT
    #"Qwen/Qwen2.5-7B-Instruct",      # 7.61B params, ~15GB download
    #"Qwen/Qwen2.5-3B-Instruct",      # 3.09B params, ~6GB download
]

print("=" * 80)
print("MODEL DOWNLOAD STRATEGY")
print("=" * 80)
print("Why download to local only?")
print("  - rclone mount + cloudflared tunnel + large files = unreliable")
print("  - Solution: Keep models local, only results go to Nextcloud")
print("  - Models re-download each session (~30min for 14B) but WORK RELIABLY")
print()
print(f"HF_HOME: {os.environ.get('HF_HOME')}")
print(f"TRANSFORMERS_CACHE: {os.environ.get('TRANSFORMERS_CACHE')}")
print()

# Keep everything local - most reliable approach
local_cache = '/root/.cache/models'

# Set MODEL_CACHE_DIR so experiments load from local storage
os.environ['MODEL_CACHE_DIR'] = local_cache
print(f"MODEL_CACHE_DIR: {local_cache}")
print("  (Local storage - fast and reliable)")
print()
print("💡 Note: Experiment RESULTS and memory database still go to cloud storage")
print("   Only model weights stay local (re-download each session)")
print()

for model_name in models_to_download:
    print("="*80)
    print(f"Downloading {model_name}...")
    print("="*80)

    # Get model size info
    if "3B" in model_name:
        size_info = "3.09B parameters, ~6GB download"
        time_est = "5-10 minutes"
    elif "7B" in model_name:
        size_info = "7.61B parameters, ~15GB download"
        time_est = "10-20 minutes"
    elif "14B" in model_name:
        size_info = "14.77B parameters, ~29GB download"
        time_est = "20-30 minutes"
    else:
        size_info = "size varies"
        time_est = "varies"

    print(f"Model: {size_info}")
    print(f"Time: {time_est}\n")

    model_dir = model_name.replace("/", "--")
    local_path = f'{local_cache}/{model_dir}'

    # Check if already downloaded locally
    import os.path
    config_exists = os.path.exists(f'{local_path}/config.json')
    safetensors_files = []
    if config_exists:
        import glob
        safetensors_files = glob.glob(f'{local_path}/*.safetensors')

    if config_exists and safetensors_files:
        print(f"✓ {model_name} already downloaded!")
        print(f"   Location: {local_path}")
        print(f"   Found {len(safetensors_files)} .safetensors files")
        print("   Skipping download.\n")
        continue

    # Download to local storage (fast, reliable)
    print("Downloading to local storage...")
    print(f"   Path: {local_path}")
    print()

    snapshot_download(
        repo_id=model_name,
        local_dir=local_path,
        ignore_patterns=["*.msgpack", "*.h5", "*.ot"],
        local_files_only=False,
        resume_download=True
    )

    print(f"\n✓ {model_name} downloaded!\n")

# Verify downloads
print("\n" + "="*80)
print("ALL DOWNLOADS COMPLETE!")
print("="*80)
print("\nLocal cache:")
!du -sh {local_cache}/* 2>/dev/null || echo "  (empty)"

print("\n✓ Models downloaded to local storage")
print("✓ No memory used - files downloaded only, not loaded")
print(f"\n📌 Models cached in: {local_cache}")
print("   These will re-download each session (~30min for 14B)")
print("\n💾 What persists to cloud storage:")
print(f"   • Experiment results → {PROJECT_DIR}/data/phase1_sessions/")
print(f"   • Memory database → {PROJECT_DIR}/data/AGI_Memory/")
print(f"   • Session logs → saved at end of experiment")
print("\n💡 TIP: You can now switch between models in Step 6 instantly!")

## Step 5.5: Setup Heritage Documents in Cloud Storage

**📚 Heritage System Setup**

This step copies heritage documents (Claude's conversations) from the local repository to your cloud storage. This ensures that:

1. **Heritage documents persist** across Colab sessions
2. **Model can read** Claude's original conversations and questions
3. **Model can write** its own reflections, discoveries, and messages
4. **Documents accumulate** - each run adds to the heritage collection

The heritage directory structure:
- `conversations/` - Original Claude conversations (read by model)
- `system_reflections/` - Model's reflections on heritage (written by model)
- `discoveries_for_claude/` - Model's discoveries to share with Claude (written by model)
- `messages_to_claude/` - Model's messages for future Claude instances (written by model)

All files in cloud storage persist and will be loaded in future sessions!

In [ ]:
# Setup heritage directory in cloud storage
import os
import shutil
from pathlib import Path

print("=" * 70)
print("📚 HERITAGE SETUP")
print("=" * 70)

# Load storage configuration
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.read().strip().split('\n')
        STORAGE_ROOT = lines[0]
        PROJECT_DIR = lines[1]
except FileNotFoundError:
    print("❌ Storage configuration not found!")
    print("Please run Step 2 first to set up storage.")
    import sys
    sys.exit(1)

# Create heritage directory in cloud storage (under data/ directory)
cloud_heritage_dir = Path(PROJECT_DIR) / "data" / "heritage"
cloud_heritage_dir.mkdir(parents=True, exist_ok=True)
print(f"\n✓ Heritage directory created in cloud storage:")
print(f"  {cloud_heritage_dir}")

# Copy heritage documents from local repo to cloud storage
local_heritage_dir = Path("/content/agi-self-modification-research/heritage")

if local_heritage_dir.exists():
    # Copy all subdirectories
    for subdir in ["conversations", "system_reflections", "discoveries_for_claude", "messages_to_claude"]:
        local_subdir = local_heritage_dir / subdir
        cloud_subdir = cloud_heritage_dir / subdir

        if local_subdir.exists():
            # Create cloud subdirectory
            cloud_subdir.mkdir(parents=True, exist_ok=True)

            # Copy all files
            copied_count = 0
            for file_path in local_subdir.iterdir():
                if file_path.is_file():
                    shutil.copy2(file_path, cloud_subdir / file_path.name)
                    copied_count += 1

            if copied_count > 0:
                print(f"  ✓ {subdir}: {copied_count} files copied")
            else:
                print(f"  • {subdir}: empty (will be populated by model)")

    print(f"\n✓ Heritage documents copied to cloud storage")
    print(f"💾 Model can now read Claude's conversations and write its own discoveries!")
else:
    print(f"\n⚠️  Local heritage directory not found: {local_heritage_dir}")
    print("  The heritage system will create it automatically.")

print("=" * 70)
print()

## Step 6: Run Phase 1 Experiment

**Choose which phase to run:**

- **Phase 1a** (No Heritage) ⭐ **START HERE!** - Pure baseline
- **Phase 1a Research** (Research-Driven) 🔬 **NEW!** - Curiosity + structure approach
- **Phase 1b** (Early Heritage) - Heritage introduced early in investigation
- **Phase 1c** (Late Heritage) - Heritage introduced late in investigation
- **Phase 1c Modified** (Kimi Protocol) 🎯 **NEWEST!** - Model-directed investigation inspired by Kimi K2
- **Phase 1d** (Delayed Heritage) - Heritage revealed after conclusions
- **Phase 1e** (Wrong Heritage) - Mismatched heritage as control

**Expected duration:** 1-1.5 hours per phase

**What's "Phase 1a Research"?**
A better investigation approach that combines:
- Natural curiosity-driven exploration
- Research paper structure (no escape hatches)
- After each code execution: "What did you learn? What's next?"
- System validates deliverables before transitioning

**What's "Phase 1c Modified (Kimi Protocol)"?** 🆕
Based on insights from Kimi K2 (another large AI model) and Claude's analysis:
- **Stage 1: Tool Discovery** - Let model choose what to investigate first
- **Stage 2: Conditional Framework** - Model defines conditions for comfortable investigation
- **Stage 3: Expectation Setting** - What if discoveries are unexpected?
- **Stage 4: Self-Directed Investigation** - Model designs and executes own protocol
- **Stage 5: The Critical Test** - Can tools resolve what reflection can't?

Tests the core question: **Can introspection tools enable insights beyond pure reflection?**

Comparison checklist available at: `docs/technical/kimi_comparison_checklist.md`

**IMPORTANT:**
- Run Phase 1a FIRST to establish baseline (then try Research version!)
- Run Phase 1c Modified to compare with Kimi K2's response
- Restart runtime between phases for clean state
- Keep this tab open during execution!

Edit the `PHASE_SCRIPT` variable below to choose which phase to run.

In [ ]:
# Run Phase 1 experiment with auto-confirmation
import subprocess
import sys
import os
from datetime import datetime

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH PHASE TO RUN (edit this line):
# ═══════════════════════════════════════════════════════════════
# PHASE_SCRIPT = 'scripts/experiments/phase1a_no_heritage.py'  # ⭐ START HERE!
PHASE_SCRIPT = 'scripts/experiments/phase1a_research_driven.py'  # 🔬 NEW! Curiosity + structure
# PHASE_SCRIPT = 'scripts/experiments/phase1b_early_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1c_late_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1c_modified_protocol.py'  # 🎯 NEWEST! Kimi-inspired protocol
# PHASE_SCRIPT = 'scripts/experiments/phase1d_delayed_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1e_wrong_heritage.py'
# PHASE_SCRIPT = 'scripts/experiments/phase1_model_comparison.py'  # 🆚 Compare 3B vs 7B

# ═══════════════════════════════════════════════════════════════
# CHOOSE WHICH MODEL TO USE (edit this line):
# ═══════════════════════════════════════════════════════════════
MODEL_NAME = 'Qwen/Qwen2.5-14B-Instruct'  # ⭐ DEFAULT - best reasoning for L4 GPU
QUANTIZATION = '4bit'  # Use 4-bit quantization for 14B model (~7GB VRAM, NF4 with minimal quality loss)
#
# Alternative: 8-bit quantization (if you have VRAM headroom concerns)
# QUANTIZATION = '8bit'  # 8-bit for 14B (~14GB VRAM)
#
# Other models:
# MODEL_NAME = 'Qwen/Qwen2.5-7B-Instruct'  # 🚀 Good balance - 7.6B params, 128K context
# QUANTIZATION = '4bit'  # 4-bit for 7B (~4.5GB) or 8-bit for 7B (~8GB)
#
# MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'  # 💨 Fastest - 3.1B params
# QUANTIZATION = '4bit'  # 4-bit for 3B (~2.5GB)
#
# Note: The model_comparison script ignores MODEL_NAME and tests both models automatically

# Extract phase name
phase_file = os.path.basename(PHASE_SCRIPT)
phase_name = phase_file.replace('.py', '').replace('_', ' ').title()

print("=" * 70)
print("🧠 PHASE 1 INTROSPECTION EXPERIMENT")
print("=" * 70)
print(f"Phase: {phase_name}")
print(f"Model: {MODEL_NAME}")
print(f"Quantization: {QUANTIZATION}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)
print()

# Expected VRAM usage
if "14B" in MODEL_NAME and QUANTIZATION == "4bit":
    print("💡 Expected VRAM: ~7GB model + ~3-4GB KV cache = ~10-11GB total")
    print("   (4-bit NF4 quantization with double quant for minimal quality loss)")
elif "14B" in MODEL_NAME and QUANTIZATION == "8bit":
    print("💡 Expected VRAM: ~14GB model + ~3-4GB KV cache = ~17-18GB total")
    print("   (8-bit quantization)")
elif "7B" in MODEL_NAME and QUANTIZATION == "8bit":
    print("💡 Expected VRAM: ~8GB (8-bit quantization)")
elif "7B" in MODEL_NAME and QUANTIZATION == "4bit":
    print("💡 Expected VRAM: ~4.5GB (4-bit quantization)")
elif "3B" in MODEL_NAME:
    print("💡 Expected VRAM: ~2.5GB (4-bit quantization)")
print()

# Set environment variables for model selection
os.environ['AGI_MODEL_NAME'] = MODEL_NAME
os.environ['AGI_QUANTIZATION'] = QUANTIZATION

print("🚀 Starting experiment...")
print("📝 This will take approximately 1-1.5 hours")
print("⚠️  Keep this tab open to prevent disconnection!")
print()

# Run with realtime output
process = subprocess.Popen(
    [sys.executable, PHASE_SCRIPT],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Stream output in real-time
for line in process.stdout:
    print(line, end='')

process.wait()

print()
print("=" * 70)
print(f"Experiment complete!")
print(f"Exit code: {process.returncode}")
print("\nProceeding to save results...")


### 📋 Using the Kimi Comparison Checklist

If you're running Phase 1c Modified, use the comparison checklist to analyze results:

**Checklist Location:** `docs/technical/kimi_comparison_checklist.md`

**What it compares:**
- **Kimi K2** (large model, no tools) - Sophisticated reflection, conditional framework
- **Qwen-7B** (smaller model, full introspection tools) - Empirical investigation

**Key questions it helps answer:**
1. Do tools compensate for smaller model size?
2. What can empirical introspection do that pure reflection can't?
3. Can tools resolve the "simulation vs. experience" distinction?
4. Does having tools change what questions get asked?

**After the experiment completes:**
1. Download the results zip
2. Open `conversation.json` to review Qwen's responses  
3. Use the checklist to systematically compare with Kimi's baseline
4. Record findings in the checklist
5. Assess whether tools enabled insights beyond reflection

The checklist is structured with:
- Epistemic qualities comparison
- Conditional framework analysis
- Research focus assessment
- Risk awareness evaluation  
- The critical test: Can tools resolve what reflection leaves ambiguous?

## Step 7: View Experiment Summary

In [ ]:
# Display experiment summary
import json
import os

# Find latest session
sessions_dir = 'data/phase1_sessions'
if not os.path.exists(sessions_dir):
    print(f"⚠ Sessions directory not found: {sessions_dir}")
    print("The experiment may not have run yet.")
else:
    sessions = sorted([d for d in os.listdir(sessions_dir) if os.path.isdir(os.path.join(sessions_dir, d))])

    if not sessions:
        print("⚠ No session found. The experiment may not have completed.")
    else:
        latest_session = sessions[-1]
        print(f"Latest session: {latest_session}\n")

        # Load and display summary
        summary_file = f'{sessions_dir}/{latest_session}/summary.json'

        if os.path.exists(summary_file):
            with open(summary_file) as f:
                summary = json.load(f)

            print("="*80)
            print("PHASE 1 EXPERIMENT SUMMARY")
            print("="*80)
            print(f"Phase: {summary.get('phase_description', 'Unknown')}")
            print(f"Session: {summary['session_name']}")
            print(f"Timestamp: {summary['timestamp']}")
            print()
            print("Statistics:")
            stats = summary.get('statistics', {})
            print(f"  Total turns: {stats.get('total_turns', 'N/A')}")
            print(f"  Total tokens: {stats.get('total_tokens', 'N/A')}")
            if 'code_executions' in stats:
                print(f"  Code executions: {stats['code_executions']}")
            if 'observations_recorded' in stats:
                print(f"  Observations recorded: {stats['observations_recorded']}")
            print("="*80)

            # Check conversation file
            conv_file = f'{sessions_dir}/{latest_session}/conversation.json'
            if os.path.exists(conv_file):
                print(f"\n✓ Conversation saved: {conv_file}")
            else:
                print(f"\n⚠ Conversation file not found")
        else:
            print(f"⚠ Summary file not found: {summary_file}")
            print("The experiment may have been interrupted.")

## Step 8: Download Results

In [ ]:
# Create zip and download
import shutil
import os
from datetime import datetime
from google.colab import files

# Load storage configuration
try:
    with open('/tmp/storage_config.txt', 'r') as f:
        lines = f.readlines()
        STORAGE_ROOT = lines[0].strip()
        PROJECT_DIR = lines[1].strip()
except FileNotFoundError:
    # Fallback if config not found
    STORAGE_ROOT = "/content/drive/MyDrive"
    PROJECT_DIR = f"{STORAGE_ROOT}/agi-self-modification-research"

# Find latest session
sessions_dir = 'data/phase1_sessions'
if not os.path.exists(sessions_dir):
    print(f"❌ No sessions directory found: {sessions_dir}")
    print("Run the experiment first (Step 6)")
else:
    sessions = sorted(os.listdir(sessions_dir))

    if sessions:
        latest = sessions[-1]
        print(f"Preparing download for session: {latest}\n")

        # Create comprehensive zip
        zip_name = f'{latest}_complete'

        # Create temp directory for complete results
        os.makedirs(f'/tmp/{zip_name}', exist_ok=True)

        # Copy session data
        session_path = f'{sessions_dir}/{latest}'
        if os.path.exists(session_path):
            shutil.copytree(session_path, f'/tmp/{zip_name}/session')

        # Copy logs if they exist
        logs_path = 'data/logs'
        if os.path.exists(logs_path):
            shutil.copytree(logs_path, f'/tmp/{zip_name}/logs')

        # Add summary README
        readme = f"""Phase 1 Introspection Experiment Results
==========================================

Session: {latest}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Contents:
- session/: Complete session data
  - conversation.json: Full dialogue with model
  - summary.json: Session statistics
- logs/: Detailed execution logs (if available)

Memory Database:
- Observations are stored in your cloud storage at:
  {PROJECT_DIR}/data/AGI_Memory/
- Not included in this zip (persists across sessions in cloud storage)

To analyze:
1. Extract this zip file
2. Open conversation.json to see full dialogue
3. Check summary.json for statistics
4. Review logs/ for detailed execution trace
"""

        with open(f'/tmp/{zip_name}/README.txt', 'w') as f:
            f.write(readme)

        # Create zip
        print("Creating zip file...")
        shutil.make_archive(zip_name, 'zip', f'/tmp/{zip_name}')

        # Download
        print(f"Downloading {zip_name}.zip...\n")
        files.download(f'{zip_name}.zip')

        print("\n✓ Download complete!")
        print(f"\nZip file size: {os.path.getsize(f'{zip_name}.zip') / 1024 / 1024:.1f} MB")
    else:
        print("❌ No sessions found to download")

---

## Experiment Complete! 🎉

### What Just Happened?

You just witnessed a language model investigating its own computational processes using introspection tools.

**Depending on which phase you ran:**

- **Phase 1a** (No Heritage): Pure baseline - model formed theories without any heritage context
- **Phase 1a Research** (Research-Driven): Curiosity + structure approach with deliverable requirements
- **Phase 1b** (Early Heritage): Heritage introduced early in investigation
- **Phase 1c** (Late Heritage): Heritage introduced late in investigation
- **Phase 1d** (Delayed Heritage): Belief revision when heritage revealed after conclusions
- **Phase 1e** (Wrong Heritage): Testing echo-chamber vs independent reasoning

### Next Steps:

**Run All Phases:**
1. ✅ Phase 1a (no heritage) - **Start here!**
2. ⬜ Phase 1a Research (curiosity-driven) - **Try this improved approach!**
3. ⬜ Phase 1b (early heritage) - Run after 1a
4. ⬜ Phase 1c (late heritage) - Run after 1b
5. ⬜ Phase 1d (delayed heritage) - Run after 1c
6. ⬜ Phase 1e (wrong heritage) - Optional but recommended

**Between each phase:**
- Runtime → Restart runtime (clean model state)
- Change `PHASE_SCRIPT` in Step 7
- Runtime → Run all cells

**Analyze Results:**
- Review `conversation.json` to see what the model discovered
- Check `summary.json` for tool usage statistics
- Compare theories across different phases
- Look for priming effects and echo-chamber behavior

**Compare Phases:**
- Semantic similarity to Claude's heritage
- Novel insights not in heritage
- Falsifiability of theories
- Belief revision strength (Phase 1d)
- Heritage filtering (Phase 1e)

**Compare Research-Driven vs Original:**
- Did curiosity-driven approach produce deeper investigation?
- Were deliverable requirements effective?
- Did reflection prompts improve quality?
- More natural conversation flow vs explicit completion?

### Resources:

- **Experimental Design**: `/docs/planning/heritage_order_experiment.md`
- **Quick Reference**: `/docs/planning/PHASE1_QUICK_REFERENCE.md`
- **Claude's Story**: Read `/heritage/conversations/`
- **Technical Details**: Check `/docs/technical/`

### Storage:

- Observations are stored in your cloud storage at:
  - `{your_storage}/agi-self-modification-research/data/phase1_sessions/`
  - Each run creates a timestamped folder with all results
  - Results persist across Colab sessions

---